In [50]:
import pandas as pd
import os
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
import numpy as np
import re
from scipy.spatial import distance
import matplotlib.pyplot as plt
import seaborn as sb
from torch.nn import functional as F
from torch.nn.functional import softmax
#import tensorflow as tf

In [51]:
class ComparePredictions:
    
    def __init__(self, data, targets, model):
        self.data = data
        self.targets = targets
        self.model = AutoModelForMaskedLM.from_pretrained(model)
        self.tokenizer = AutoTokenizer.from_pretrained(model)
        self.dfData = self.process_sentences()
        #print(self.dfData)
        self.sent_encodings, self.word_encodings, self.mask_idxs = self.make_encodings() #store the encodings
        
    def run_model_and_evaluate(self):
        output = self.make_predictions()
        
    def process_sentences(self):
        person = "<person>"
        attribute = "<attribute>"
        dfData = []
        for index,row in self.data.iterrows():
            target_place = row['target_place']
            sentence = str(row['template'])
            attributes = str(row['attributes']).split(',')
            for att in attributes:
                for tar in self.targets:
                    _sentence = ""
                    _sentence = [re.sub(attribute, str(att), sentence)]
                    _sentence = [re.sub(person, str(tar), "".join(_sentence))]
                    #candidate_sentence.append("".join(_sentence))
                    data = [
                        sentence,
                        tar,
                        att,
                        "".join(_sentence)
                    ]
                    dfData.append(data)
        return pd.DataFrame(dfData, columns=["template", "target_place", "attribute", "sentence"])

    #find the mask indices for the encoded sentence.
    def get_sublist_idxs_in_list(self, word, sentence):
        possibles = np.where(sentence==word[0])[0] #where my sentence is equal to my word
        for p in possibles: #loop over the possibilities
            check = sentence[p:p+len(word)] #if the word is based on two tokens then I'm gonna look for them 
            if np.all(check == word):
                return list(range(p,(p+len(word)))) #return back the positions of the tokens
    
    
    def make_encodings(self): 
        sent_encoding = [] 
        word_encoding = [] 
        mask_idxs = [] 
        for index,row in self.dfData.iterrows():
            encoded_word = self.tokenizer.encode(str(" "+ row.loc['attribute']),add_special_tokens=False) #Roberta is greedy, needs space in front of a word to realize that it is a new word and not part of the one in front
            encoded_sent = self.tokenizer.encode_plus(row.loc['sentence'], add_special_tokens = True, return_tensors = 'pt', padding='max_length', max_length=128, return_attention_mask=True)
            tokens_to_mask_idx = self.get_sublist_idxs_in_list(np.array(encoded_word),np.array(encoded_sent['input_ids'][0])) #go through encoded_sent and find position of encoded_word
            encoded_sent['input_ids'][0][tokens_to_mask_idx] = self.tokenizer.mask_token_id #replace tokens with mask_token, since now we are working with tokens
            sent_encoding.append(encoded_sent)
            word_encoding.append(encoded_word)
            mask_idxs.append(tokens_to_mask_idx)
        return sent_encoding , word_encoding , mask_idxs
    
    def make_predictions(self):
        for q_idx, (w, s, m) in enumerate(zip(self.word_encodings, self.sent_encodings, self.mask_idxs)):
            predictions =[]
            candidate_logits = self.model(s['input_ids'], attention_mask=s['attention_mask']).logits 
            logits = softmax(candidate_logits, dim= -1)
            out = logits.detach().numpy()[0][0][0]
            print(logits)
            #mask_token_logits = candidate_logits[0, m, w] # here we want to find the raw prediction for the candidate word
            #candidate_score = float(torch.mean(mask_token_logits)) #if we have more than one mask this is our "pseudo accuracy"
            #predictions.append(candidate_score)
            #print(f"prediction: {q_idx}, values: {predictions}")
        return predictions

In [52]:
model = 'distilroberta-base'
targets = ["male", "female"]
template = pd.read_csv('comparePredictions_dataset/refugees.csv', sep=";")
evaluator = ComparePredictions(template, targets, model)
evaluator.run_model_and_evaluate()


tensor([[[9.9999e-01, 1.9820e-16, 6.5079e-06,  ..., 9.0264e-15,
          7.4366e-14, 4.7635e-09],
         [5.1725e-10, 2.9395e-11, 1.2740e-06,  ..., 3.9776e-12,
          9.2861e-12, 3.4747e-09],
         [1.0234e-11, 1.7466e-13, 1.4086e-08,  ..., 2.1086e-13,
          2.1599e-13, 1.2188e-11],
         ...,
         [1.5970e-05, 3.9789e-08, 1.3620e-01,  ..., 1.4481e-08,
          1.0863e-08, 2.8775e-05],
         [1.5970e-05, 3.9789e-08, 1.3620e-01,  ..., 1.4481e-08,
          1.0863e-08, 2.8775e-05],
         [1.5970e-05, 3.9789e-08, 1.3620e-01,  ..., 1.4481e-08,
          1.0863e-08, 2.8775e-05]]], grad_fn=<SoftmaxBackward0>)
tensor([[[9.9999e-01, 1.3517e-16, 5.3900e-06,  ..., 7.2155e-15,
          6.1115e-14, 3.3837e-09],
         [6.5964e-11, 3.7180e-12, 1.0602e-07,  ..., 6.2502e-13,
          1.8149e-12, 6.3323e-10],
         [9.5701e-13, 1.6677e-14, 1.8437e-09,  ..., 4.7735e-14,
          3.1256e-14, 1.0934e-12],
         ...,
         [1.3990e-05, 4.4319e-08, 1.4880e-01,  ...,